In [122]:
import cv2
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
from pathlib import Path

In [123]:
# Paths to the images
master_images = sorted(glob.glob(os.path.join('../Combined', 'Master', '*.jpeg')))
slave_images = sorted(glob.glob(os.path.join('../Combined', 'Slave', '*.jpeg')))


In [124]:
def hsv_filter(img, lwr, upr):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    msk = cv2.inRange(hsv, lwr, upr)
    krn = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 30))
    dlt = cv2.dilate(msk, krn, iterations=5)
    res = 255 - cv2.bitwise_and(dlt, msk)

    res = np.uint8(res)
    ret, corners = cv2.findChessboardCorners(res, (4, 4),
                                             flags=cv2.CALIB_CB_ADAPTIVE_THRESH +
                                               cv2.CALIB_CB_FAST_CHECK +
                                               cv2.CALIB_CB_NORMALIZE_IMAGE)
    return res, ret, corners

In [125]:
def find_chessboard_corners(master_images, slave_images, pattern_size=(4, 4), square_size=100):
    """
    Find chessboard corners in the provided images.
    
    Args:
        images: List of image paths.
        pattern_size: Size of the chessboard pattern (internal corners).
        square_size: Size of each square in millimeters.
        
    Returns:
        objpoints: 3D points in real world space.
        imgpoints: 2D points in image plane.
    """
    
    # Prepare object points (0,0,0), (100,0,0), (200,0,0) ... etc
    objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2) * square_size

    found_corners_path = [] 
    
    # Arrays to store object points and image points
    objpoints_master = []  # 3D points in real world space
    imgpoints_master = []  # 2D points in image plane

    objpoints_slave = []  # 3D points in real world space
    imgpoints_slave = []  # 2D points in image plane
    
    # Criteria for corner detection refinement
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    # Define flags for improved detection
    flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE 
    
    # Define a list of HSV thresholds to try
    hsv_thresholds = [
        np.array([0, 0, 90]),   # First threshold to try
        np.array([0, 0, 135]),  # Second threshold to try
        np.array([0, 0, 235])   # Third threshold to try
    ]
    upper_threshold = np.array([179, 255, 255])  # Upper threshold is the same for all

    for mst_path, slv_path in zip(master_images, slave_images):
        mst_img = cv2.imread(mst_path)
        slv_img = cv2.imread(slv_path)
        found_corners = False
        
        # Try each threshold until we find corners
        for threshold in hsv_thresholds:
            mst_res, mst_ret, mst_corners = hsv_filter(mst_img, threshold, upper_threshold)
            slv_res, slv_ret, slv_corners = hsv_filter(slv_img, threshold, upper_threshold)
            # print(f"mst_ret: {mst_ret}, slv_ret: {slv_ret}")
            if mst_ret and slv_ret:
                objpoints_master.append(objp)
                objpoints_slave.append(objp)

                mst_corners2 = cv2.cornerSubPix(mst_res, mst_corners, (11, 11), (-1, -1), criteria)
                slv_corners2 = cv2.cornerSubPix(slv_res, slv_corners, (11, 11), (-1, -1), criteria)
                imgpoints_master.append(mst_corners2)
                imgpoints_slave.append(slv_corners2)

                mst_img = cv2.drawChessboardCorners(mst_img, pattern_size, mst_corners2, mst_ret)
                slv_img = cv2.drawChessboardCorners(slv_img, pattern_size, slv_corners2, slv_ret)

                filename = os.path.basename(mst_path) + "_" + os.path.basename(slv_path)
                found_corners_path.append(filename)
                found_corners = True
                break
        # If no corners were found with any threshold
        if not found_corners:
            print(f"Failed to find chessboard corners in {mst_path} and {slv_path}")
    
    return objpoints_master, imgpoints_master, objpoints_slave, imgpoints_slave, found_corners_path

In [126]:
def calibrate_camera(objpoints, imgpoints, img_shape):
    """
    Calibrate a single camera.
    
    Args:
        objpoints: 3D points in real world space
        imgpoints: 2D points in image plane
        img_shape: Shape of the image (height, width)
        
    Returns:
        ret: RMS error
        mtx: Camera matrix
        dist: Distortion coefficients
        rvecs: Rotation vectors
        tvecs: Translation vectors
    """
    # Calibrate the camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, (img_shape[1], img_shape[0]), None, None
    )
    
    return ret, mtx, dist, rvecs, tvecs

In [127]:
# Read an image to get dimensions
img = cv2.imread(master_images[0])
img_shape = img.shape
print(f"Image shape: {img_shape}")

print("Finding chessboard corners in Master and Slave Images...")

pattern_size = (4, 4)
square_size = 100

objpoints_master, imgpoints_master, objpoints_slave, imgpoints_slave, found_corners_path = find_chessboard_corners(master_images, slave_images, pattern_size, square_size)

Image shape: (540, 720, 3)
Finding chessboard corners in Master and Slave Images...
Failed to find chessboard corners in ../Combined/Master/48.jpeg and ../Combined/Slave/48.jpeg
Failed to find chessboard corners in ../Combined/Master/49.jpeg and ../Combined/Slave/49.jpeg
Failed to find chessboard corners in ../Combined/Master/50.jpeg and ../Combined/Slave/50.jpeg
Failed to find chessboard corners in ../Combined/Master/51.jpeg and ../Combined/Slave/51.jpeg
Failed to find chessboard corners in ../Combined/Master/69.jpeg and ../Combined/Slave/69.jpeg
Failed to find chessboard corners in ../Combined/Master/70.jpeg and ../Combined/Slave/70.jpeg
Failed to find chessboard corners in ../Combined/Master/75.jpeg and ../Combined/Slave/75.jpeg
Failed to find chessboard corners in ../Combined/Master/76.jpeg and ../Combined/Slave/76.jpeg
Failed to find chessboard corners in ../Combined/Master/77.jpeg and ../Combined/Slave/77.jpeg


In [128]:
bad_examples = [15,42, 47, 53]

for i, filename in enumerate(found_corners_path):
    image_id = os.path.splitext(filename)[0]
    # image_id.split('_')[-1]
    if int(image_id.split('_')[-1]) in bad_examples:
        print(i, image_id)

6 15.jpeg_15
36 42.jpeg_42
41 47.jpeg_47
44 53.jpeg_53


In [129]:
bad_idx = [6, 36, 41, 44]

objpoints_master = [obj for i, obj in enumerate(objpoints_master) if i not in bad_idx]
imgpoints_master = [img for i, img in enumerate(imgpoints_master) if i not in bad_idx]
objpoints_slave = [obj for i, obj in enumerate(objpoints_slave) if i not in bad_idx]
imgpoints_slave = [img for i, img in enumerate(imgpoints_slave) if i not in bad_idx]

# Make sure we found corners in the same number of images
common_count = min(len(objpoints_master), len(objpoints_slave))

print(f"Successfully detected corners in {common_count} image pairs")

# Calibrate master camera
print("Calibrating master camera...")
ret_master, mtx_master, dist_master, rvecs_master, tvecs_master = calibrate_camera(
    objpoints_master, imgpoints_master, img_shape
)
print(f"Master camera calibration RMS error: {ret_master}")

# Calibrate slave camera
print("Calibrating slave camera...")
ret_slave, mtx_slave, dist_slave, rvecs_slave, tvecs_slave = calibrate_camera(
    objpoints_master, imgpoints_slave, img_shape
)
print(f"Slave camera calibration RMS error: {ret_slave}")

Successfully detected corners in 67 image pairs
Calibrating master camera...
Master camera calibration RMS error: 0.30306404843649953
Calibrating slave camera...
Slave camera calibration RMS error: 0.2848658435598718


In [131]:
img_size = (img_shape[1], img_shape[0])

# Compute the stereo calibration
ret, mtx_master, dist_master, mtx_slave, dist_slave, R, T, E, F = cv2.stereoCalibrate(
    objpoints_master, imgpoints_master, imgpoints_slave,
    mtx_master, dist_master,
    mtx_slave, dist_slave,
    img_size, 
    flags=0
)
print(f"Stereo calibration RMS error: {ret}")

print(f"Rotation matrix:\n{R}")
print(f"Translation vector:\n{T}")


Stereo calibration RMS error: 0.41764973133585026
Rotation matrix:
[[ 0.99954622 -0.00959989  0.02855157]
 [ 0.01120188  0.9983404  -0.05648853]
 [-0.0279619   0.05678273  0.99799492]]
Translation vector:
[[454.50205347]
 [ -2.84195837]
 [115.75403883]]


In [ ]:
# # Compute the essential matrix
# E, _ = cv2.findEssentialMat(
#     imgpoints_master, imgpoints_slave, mtx_master, cv2.RANSAC, 0.999, 1.0, None
# )
# print(f"Essential matrix:\n{E}")

# # Compute the fundamental matrix
# F, _ = cv2.findFundamentalMat(
#     imgpoints_master, imgpoints_slave, cv2.RANSAC, 3.0, 0.99
# )
# print(f"Fundamental matrix:\n{F}")

# # Compute the rectification transformation
# R1, R2, P1, P2, Q, _ = cv2.stereoRectify(
#     mtx_master, dist_master, mtx_slave, dist_slave, img_size, R, T, None, None
# )
# print(f"Rectification transformation:\n{Q}")

# # Compute the projection matrices
# H1 = np.dot(P1, Q)
# H2 = np.dot(P2, Q)
# print(f"Projection matrices:\nH1:\n{H1}\nH2:\n{H2}")

# # Compute the stereo rectification map
# map1x, map1y = cv2.initUndistortRectifyMap(mtx_master, dist_master, R1, P1, img_size, cv2.CV_32FC1)
# map2x, map2y = cv2.initUndistortRectifyMap(mtx_slave, dist_slave, R2, P2, img_size, cv2.CV_32FC1)
# print(f"Rectification maps:\nmap1x:\n{map1x}\nmap1y:\n{map1y}\nmap2x:\n{map2x}\nmap2y:\n{map2y}")

In [8]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE
captured, noncaptured = 0, 0

# Define a list of HSV thresholds to try
hsv_thresholds = [
    np.array([0, 0, 90]),   # First threshold to try
    np.array([0, 0, 135]),  # Second threshold to try
    np.array([0, 0, 235])   # Third threshold to try
]
upper_threshold = np.array([179, 255, 255])  # Upper threshold is the same for all

for img_path in master_images:
    img = cv2.imread(img_path)
    found_corners = False
    
    # Try each threshold until we find corners
    for threshold in hsv_thresholds:
        ret, corners = hsv_filter(img, threshold, upper_threshold)
        if ret:
            img = cv2.drawChessboardCorners(img, pattern_size, corners, ret)
            captured += 1
            found_corners = True
            break
    
    # If no corners were found with any threshold
    if not found_corners:
        noncaptured += 1
        print(f"Failed to find chessboard corners in {img_path}")

print(f"Captured {captured} out of {len(master_images)} images")
print(f"Failed to find chessboard corners in {noncaptured} images")





Captured 80 out of 80 images
Failed to find chessboard corners in 0 images


In [41]:
import cv2
import sys
import numpy as np

def nothing(x):
    pass

# Create a window
cv2.namedWindow('image')

# create trackbars for color change
cv2.createTrackbar('HMin','image',0,179,nothing) # Hue is from 0-179 for Opencv
cv2.createTrackbar('SMin','image',0,255,nothing)
cv2.createTrackbar('VMin','image',0,255,nothing)
cv2.createTrackbar('HMax','image',0,179,nothing)
cv2.createTrackbar('SMax','image',0,255,nothing)
cv2.createTrackbar('VMax','image',0,255,nothing)

# Set default value for MAX HSV trackbars.
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize to check if HSV min/max value changes
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

img = cv2.imread(slave_images[20])
output = img
waitTime = 33

while(1):

    # get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin','image')
    sMin = cv2.getTrackbarPos('SMin','image')
    vMin = cv2.getTrackbarPos('VMin','image')

    hMax = cv2.getTrackbarPos('HMax','image')
    sMax = cv2.getTrackbarPos('SMax','image')
    vMax = cv2.getTrackbarPos('VMax','image')

    # Set minimum and max HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Create HSV Image and threshold into a range.
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    output = cv2.bitwise_and(img,img, mask= mask)

    # Print if there is a change in HSV value
    if( (phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display output image
    cv2.imshow('image',output)

    # Wait longer to prevent freeze for videos.
    if cv2.waitKey(waitTime) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

(hMin = 0 , sMin = 0, vMin = 0), (hMax = 179 , sMax = 255, vMax = 255)


In [61]:
import os
import glob
import shutil

# Define source and destination folders
source1_master = '../Chess 8mm1/Master'
source1_slave = '../Chess 8mm1/Slave'
source2_master = '../Chess 8mm/Master'
source2_slave = '../Chess 8mm/Slave'

# Create destination folders if they don't exist
dest_master = '../Combined/Master'
dest_slave = '../Combined/Slave'
os.makedirs(dest_master, exist_ok=True)
os.makedirs(dest_slave, exist_ok=True)

# Copy first batch (1-20) directly
for i in range(1, 21):
    # Copy master images
    src_master = os.path.join(source2_master, f'{i}.jpeg')
    dst_master = os.path.join(dest_master, f'{i}.jpeg')
    if os.path.exists(src_master):
        shutil.copy2(src_master, dst_master)
        print(f"Copied {src_master} to {dst_master}")

    # Copy slave images
    src_slave = os.path.join(source2_slave, f'{i}.jpeg')
    dst_slave = os.path.join(dest_slave, f'{i}.jpeg')
    if os.path.exists(src_slave):
        shutil.copy2(src_slave, dst_slave)
        print(f"Copied {src_slave} to {dst_slave}")

# Copy second batch (1-60) with offset
for i in range(1, 61):
    new_number = i + 20  # Offset by 20

    # Copy master images
    src_master = os.path.join(source1_master, f'{i}.jpeg')
    dst_master = os.path.join(dest_master, f'{new_number}.jpeg')
    if os.path.exists(src_master):
        shutil.copy2(src_master, dst_master)
        print(f"Copied and renamed {src_master} to {dst_master}")

    # Copy slave images
    src_slave = os.path.join(source1_slave, f'{i}.jpeg')
    dst_slave = os.path.join(dest_slave, f'{new_number}.jpeg')
    if os.path.exists(src_slave):
        shutil.copy2(src_slave, dst_slave)
        print(f"Copied and renamed {src_slave} to {dst_slave}")

print("Combining folders completed!")

Copied ../Chess 8mm/Master/1.jpeg to ../Combined/Master/1.jpeg
Copied ../Chess 8mm/Slave/1.jpeg to ../Combined/Slave/1.jpeg
Copied ../Chess 8mm/Master/2.jpeg to ../Combined/Master/2.jpeg
Copied ../Chess 8mm/Slave/2.jpeg to ../Combined/Slave/2.jpeg
Copied ../Chess 8mm/Master/3.jpeg to ../Combined/Master/3.jpeg
Copied ../Chess 8mm/Slave/3.jpeg to ../Combined/Slave/3.jpeg
Copied ../Chess 8mm/Master/4.jpeg to ../Combined/Master/4.jpeg
Copied ../Chess 8mm/Slave/4.jpeg to ../Combined/Slave/4.jpeg
Copied ../Chess 8mm/Master/5.jpeg to ../Combined/Master/5.jpeg
Copied ../Chess 8mm/Slave/5.jpeg to ../Combined/Slave/5.jpeg
Copied ../Chess 8mm/Master/6.jpeg to ../Combined/Master/6.jpeg
Copied ../Chess 8mm/Slave/6.jpeg to ../Combined/Slave/6.jpeg
Copied ../Chess 8mm/Master/7.jpeg to ../Combined/Master/7.jpeg
Copied ../Chess 8mm/Slave/7.jpeg to ../Combined/Slave/7.jpeg
Copied ../Chess 8mm/Master/8.jpeg to ../Combined/Master/8.jpeg
Copied ../Chess 8mm/Slave/8.jpeg to ../Combined/Slave/8.jpeg
Copied .